In [23]:
def filtrate_data(file,pressure,path): 
    
    #This function only works to treat data from the small unit operation
    #In order for the function to work correctly, the data should be collected with the time intervals set to the time where the first experiment of the day started, and to the time where the last experiment of the day ended
    #This function only works for operations with constant pressure
    
    
    import pandas as pd
    import datetime
    import os
    
    #Treating the data considering specifically the format in which the data from the small filtration unit comes
    df=pd.read_excel(file)
    df.drop(0)
    df = df.drop(df.columns[(df.iloc[1] == 'Status') | (df.iloc[1] == 'Limit')], axis = 1)
    df = df.drop(df.columns[(df.iloc[0] == 'Status') | (df.iloc[0] == 'Limit')], axis = 1)
    df = df.rename(columns={'Group 1': 'Date', 'Unnamed: 1': 'Time', 'Unnamed: 5': 'PIR01-p vor Mem.', 'Unnamed: 8': 'PIR02-p nach Mem.', 'Unnamed: 11': 'FIR01- Konz.', 'Unnamed: 14': 'FIR02-Filtr./Per', 'Unnamed: 17': 'TIR01 Membran'})
    df=df.drop([0,1,2])
    df = df.replace(',', '.', regex=True)
    df['datetime'] = pd.to_datetime(df['Date'] + ' ' + df['Time'].astype(str), format='%d.%m.%Y %H:%M:%S')
    df.set_index('datetime', inplace=True)
    df[['PIR01-p vor Mem.', 'PIR02-p nach Mem.', 'FIR01- Konz.', 'FIR02-Filtr./Per', 'TIR01 Membran']] = df[['PIR01-p vor Mem.', 'PIR02-p nach Mem.', 'FIR01- Konz.', 'FIR02-Filtr./Per', 'TIR01 Membran']].apply(pd.to_numeric, errors='coerce')
    df_60s = df.resample('60S').agg({'PIR01-p vor Mem.': 'mean', 'PIR02-p nach Mem.': 'mean', 'FIR01- Konz.' : 'mean', 'FIR02-Filtr./Per' : 'mean', 'TIR01 Membran' : 'mean'})
    
    min_press, max_press = pressure-pressure/4, pressure+pressure/4
    df_list = []
    temp_df = pd.DataFrame()
    df_60s = df_60s.fillna(0)
    
    for index, row in df_60s.iterrows():
        if index==df_60s.index[-1]:
            if min_press <= row['PIR01-p vor Mem.'] <= max_press:
                temp_df = pd.concat([temp_df, row.to_frame().T])
                df_list.append(temp_df)
        elif min_press <= row['PIR01-p vor Mem.'] <= max_press:
            temp_df = pd.concat([temp_df, row.to_frame().T])
        else:
             temp_df = pd.concat([temp_df, row.to_frame().T])
             if temp_df.shape[0] > 1:
                df_list.append(temp_df)
             temp_df = pd.DataFrame()
            
    date = str(df_60s.reset_index()['datetime'][0])
    dat = date.split()
    new_dir = 'Experiments '+dat[0]
    new_path = os.path.join(path, new_dir)
    os.mkdir(new_path)

    for i in range(len(df_list)):
        name=(str(df_list[i].reset_index()['index'][0]).split())[1].replace(':','-')
        file_name = 'Experiment {}.xlsx'.format(name)
        file_path = os.path.join(new_path, file_name)
        df_list[i].to_excel(file_path)

In [25]:
filtrate_data('0102.xlsx', 12.5, 'C:\\Users\\FilipeGomes(1180836)\\Desktop')